In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

In [ ]:
actor输出是linear层 在get_reward里normlize  

In [2]:
def get_reward(state, action):
    via_point= np.zeros(3)
    via_point[0]=action[0]*8
    via_point[1]=action[1]*8
    via_point[2]=action[2]

    old_promp=promp
    via_point = via_point.reshape([-1,3])
    t_cond = np.zeros(2+via_point.shape[0])
    t_cond[0]=0
    t_cond[-1] = 1 
    for i in range(via_point.shape[0]):
        t_cond[i+1]=via_point[i,2]

    q_cond =np.zeros([2+via_point.shape[0],2])
    q_cond[0]= np.array([state[0],state[1]])
    q_cond[-1]=np.array([state[2],state[3]])
    for i in range(via_point.shape[0]):
        q_cond[i+1]=via_point[i,:2]
        
    mu_w_cond_rec, Sigma_w_cond_rec=old_promp.get_basis_weight_parameters()

    for i in range(t_cond.shape[0]):
        mu_w_cond_rec, Sigma_w_cond_rec = old_promp.get_conditioned_weights(t_cond[i], q_cond[i], mean_w=mu_w_cond_rec, var_w=Sigma_w_cond_rec)

    cond_traj = np.zeros([2,100])
    sigma_con_array =np.zeros([2,2,100]) 
    for i in range(len(domain)):
        mu_marg_q_con, Sigma_marg_q_con = promp.get_marginal(domain[i], mu_w_cond_rec, Sigma_w_cond_rec)
        sigma_con_array[:,:,i] = Sigma_marg_q_con
        cond_traj[:,i] = mu_marg_q_con

    limit = np.array([[state[4],state[4]+1],[state[5],state[5]+1]])   
    obs_dis= traj_rect_dist(cond_traj.T,  limit)  #new distance
    orig_obs= traj_rect_dist(mean_margs.T,  limit) #old distance
    obs_reward=obs_dis- orig_obs #new distance -old distance
    # mu_1= mean_margs.T.reshape(-1)
    mu_2= cond_traj.T.reshape(-1)
    # sigma_1 = np.zeros([200,200])
    sigma_2 = np.zeros([200,200])
    for i in range(100):
        # sigma_1[i*2:(i+1)*2,i*2:(i+1)*2] = sigma_s[:,:,i]
        sigma_2[i*2:(i+1)*2,i*2:(i+1)*2] = sigma_con_array[:,:,i]
    fid_cost= calculate_frechet_distance(mu_1,sigma_1,mu_2,sigma_2) *0.01

    alif =0.1

    reward =    obs_dis*(1-alif) -alif* fid_cost
    # reward =    obs_dis
    # reward = obs_reward
    # reward =   - fid_cost
    # reward =    obs_reward*(1-alif) -alif* fid_cost
    return reward


In [128]:
from torch.distributions.multivariate_normal import MultivariateNormal


class Actor(nn.Module):
    def __init__(self, n_states, n_actions):
        super(Actor, self).__init__()
        self.stds = torch.Tensor([ 0.1,  0.1, 0.01])
      
        self.actor = nn.Sequential(
            nn.Linear(n_states, 8),
            nn.Tanh(),
            nn.Linear(8, 8),
            nn.Tanh(),
            nn.Linear(8, n_actions),
            nn.Sigmoid()
            )

    def forward(self, state):
        # out = layers.Lambda(get_range, arguments={'maxx': 30, 'minn': -20})(sigmoid_output)
        mu= self.actor(state)
        k=torch.Tensor([10,10,1])
        dist = MultivariateNormal(loc=mu*k,scale_tril=torch.diag(self.stds) )
#         dist = torch.distributions.Normal(loc=mu*k, scale=self.stds)
#         dist= dist.sample() 
        return dist

In [129]:
act_0 = Actor(6,3)
state= torch.tensor([1,2,3,4,5,6],dtype=torch.float)



# state = torch.tensor(a, dtype=torch.float).to('cpu') 
dist=act_0(state)
print(dist)

MultivariateNormal(loc: torch.Size([3]), scale_tril: torch.Size([3, 3]))


In [130]:
action = dist.sample() 
prob = dist.log_prob(action)
print(action)
print(prob)

tensor([5.5867, 6.2083, 0.5637])
tensor(6.0876, grad_fn=<SubBackward0>)


In [132]:
import torch
print(torch.__version__)#查看torch版本

print(torch.version.cuda)#查看cuda版本

print(torch.cuda.is_available())#查看cuda是否可用
#####返回True说明配置成功
print(torch.cuda.device_count())#查看可用cuda数量


1.13.0+cu117
11.7
False
1


/home/zhiyuan/anaconda3/envs/base_env/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [127]:
prob_list=[]
prob_list.append(prob)
prob_list.append(prob)
prob_list.append(prob)

old_probs_arr=np.array(prob_list)


batch = np.array([0,1])
old_probs = torch.tensor(old_probs_arr[batch]).to(self.device)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
dist = MultivariateNormal(mean, self.cov_mat)

In [ ]:
from torch.distributions.multivariate_normal import MultivariateNormal
dist = MultivariateNormal(loc=mu*k,scale_tril=torch.diag(self.stds) )

In [90]:
class PPOmemory:
    def __init__(self, mini_batch_size):
        self.states = []  # 状态
        self.actions = []  # 实际采取的动作
        self.probs = []  # 动作概率
        self.vals = []  # critic输出的状态值
        self.rewards = []  # 奖励
        self.dones = []  # 结束标志

        self.mini_batch_size = mini_batch_size  # minibatch的大小

    def sample(self):
        n_states = len(self.states)  # memory记录数量=20
        batch_start = np.arange(0, n_states, self.mini_batch_size)  # 每个batch开始的位置[0,5,10,15]
        indices = np.arange(n_states, dtype=np.int64)  # 记录编号[0,1,2....19]
        np.random.shuffle(indices)  # 打乱编号顺序[3,1,9,11....18]
        mini_batches = [indices[i:i + self.mini_batch_size] for i in batch_start]  # 生成4个minibatch，每个minibatch记录乱序且不重复

        return np.array(self.states), np.array(self.actions), np.array(self.probs), \
               np.array(self.vals), np.array(self.rewards), np.array(self.dones), mini_batches

    # 每一步都存储trace到memory
    def push(self, state, action, prob, val, reward, done):
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(prob)
        self.vals.append(val)
        self.rewards.append(reward)
        self.dones.append(done)

    # 固定步长更新完网络后清空memory
    def clear(self):
        self.states = []
        self.actions = []
        self.probs = []
        self.vals = []
        self.rewards = []
        self.dones = []


In [91]:
ppomemory=PPOmemory(5)

In [93]:
n_states = 20  # memory记录数量=20
batch_start = np.arange(0, 20, 5)  # 每个batch开始的位置[0,5,10,15]
indices = np.arange(20, dtype=np.int64)  # 记录编号[0,1,2....19]
np.random.shuffle(indices)  # 打乱编号顺序[3,1,9,11....18]
mini_batches = [indices[i:i + 5] for i in batch_start]

In [94]:
mini_batches

[array([18,  5,  7,  0,  2]),
 array([ 6, 10,  1, 19,  4]),
 array([ 8,  3, 12, 16, 13]),
 array([14,  9, 15, 11, 17])]

In [96]:
a = torch.ones(3,4)

b = torch.Tensor([1,1,3,4])

a*b



tensor([[1., 1., 3., 4.],
        [1., 1., 3., 4.],
        [1., 1., 3., 4.]])

In [108]:
via_point= np.zeros(3)
via_point = via_point.reshape([-1,3])

In [109]:
q_cond =np.zeros([2+via_point.shape[0],2])

In [110]:
q_cond.shape

(3, 2)

In [111]:
via_point.shape

(1, 3)

In [112]:
via_point[0,:2]

array([0., 0.])